### `Conv2DPytTestGroups` code

In [8]:
import numpy as np

In [80]:
class Conv2DPytTestGroups:
    ''' Computes convolution given the input parameters '''
    
    ''' * The class implementation will be along the lines of torch.nn.Conv2D in order to 
          enable comparison of this NumPy only implementation and seamless testing
        * Can expect extensive refactoring of the existing code in the days to come
        * As part of refactoring, some code will be de-modularized
        * Old code will be retained at the end of the notebook for reference
    '''
    '''
        TODO:
        * Add various `padding` modes - {'valid' (no padding / padding = 0), 'same'}
        * Add various `kernel_size` modes - a single `int` or a `tuple` of ints
        * Implementing other features and caveats offered by nn.torch.Conv2D 
          (e.g., uniform sampling of kernel weights etc.)
        * Clean-up `debug` mode
        * Optimizing code
    '''
    
    def __init__(
        self, 
        in_channels, 
        out_channels, 
        kernel_size, 
        padding = 0, 
        stride = 1, 
        dilation = 1, 
        groups = 1, 
        bias = True, 
        padding_mode = 'zeros', 
        device = None, 
        dtype = None, 
        verbose = True, 
        debug = False
        ):
        super(Conv2DPytTestGroups, self).__init__()
        
        ''' mandatory parameters '''
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        
        ''' optional parameters '''
        self.padding = padding
        self.stride = stride
        self.dilation = dilation
        self.groups = groups
        
        ''' optional parameters (dummy, yet to be implemented)'''
        self.bias = bias
        self.padding_mode = padding_mode
        self.device = device
        self.dtype = dtype
        
        ''' additional parameters (different from torch.nn.Conv2D)'''
        self.verbose = verbose
        self.verboseprint = print if self.verbose else lambda *a, **k: None
        self.debug = debug
        self.debugprint = print if self.debug else lambda *a, **k: None
        self.verboseprint('*** parameters ***')
        self.verboseprint('in_channels: {}, out_channels: {}, kernel_size: {}'.format(self.in_channels, self.out_channels, self.kernel_size))
        self.verboseprint('padding: {}, stride: {}, dilation factor: {}'.format(self.padding, self.stride, self.dilation))
        self.verboseprint('groups: {}, bias: {}, padding_mode: {}, device: {}, dtype: {}'.format(self.groups, self.bias, self.padding_mode, self.device, self.dtype))
        self.verboseprint('\n')
    
    def forward(self, _input, kernels):
        ''' forward pass to perform convolution '''
        
        ''' do error checking '''
        _input_n, _input_c, _input_h, _input_w = _input.shape
        if _input_h + 2 * self.padding < self.dilation * (self.kernel_size[0] - 1) + 1: # check if (dilated) ker_h is valid
            raise Exception('invalid input parameters: kernel height is larger than input height')
        if _input_w + 2 * self.padding < self.dilation * (self.kernel_size[1] - 1) + 1: # check if (dilated) ker_w is valid
            raise Exception('invalid input parameters: kernel width is larger than input width')
        if ((_input_h + 2 * self.padding - (self.dilation * (self.kernel_size[0] - 1) + 1)) / self.stride) + 1 < 0: # check if out_h is valid
            raise Exception('invalid input parameters: output height is negative')
        if ((_input_w + 2 * self.padding - (self.dilation * (self.kernel_size[1] - 1) + 1)) / self.stride) + 1 < 0: # check if out_w is valid
            raise Exception('invalid input parameters: output width is negative')
        if  self.in_channels % self.groups != 0: # check if groups is valid
            raise Exception('invalid input parameters: input channels is not divisible by groups')
        if self.out_channels % self.groups != 0: # check if groups is valid
            raise Exception('invalid input parametes: output channels is not divisible by groups')
         
        ''' add zero padding based on the input parameters '''
        if self.padding != 0:
            _input = np.array([[np.pad(channel,self.padding, 'constant', constant_values = 0) for channel in batch] for batch in _input])    
            self.verboseprint('*** padded input image ***')
            self.verboseprint('input batches: {}, input channels: {}, input height: {}, input weight: {}'.format(_input.shape[0], _input.shape[1], _input.shape[2], _input.shape[3]))
            self.verboseprint(_input)
            self.verboseprint('\n')
        if self.debug:
            for b in range(_input.shape[0]):
                _input[b] = (b+1) * np.ones_like(_input[b]) # define an image of all ones (twos etc.) based on the input parameters
        
        ''' use the provided kernels or create random kernels based on the input kernel parameters '''
        if kernels is not None:
            self.verboseprint('*** kernels ***')
            self.verboseprint('kernels: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]))
        else:
            kernels = []
            self.verboseprint('*** kernels ***')
            self.verboseprint('kernels: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]))
            for k in range(self.out_channels):
                kernel = np.random.rand(int(self.in_channels/self.groups), self.kernel_size[0], self.kernel_size[1]) # define a random kernel based on the kernel parameters
                if self.debug:
                    kernel = k * np.ones_like(kernel)
                kernels.append(kernel)
                self.verboseprint('kernel {}'.format(k))
                self.verboseprint(kernel)
            self.verboseprint('\n')
        
        ''' dilate a kernel '''
        dil_ker_h = self.dilation * (self.kernel_size[0] - 1) + 1
        dil_ker_w = self.dilation * (self.kernel_size[1] - 1) + 1
        dil_kernels = []
        for kernel in kernels:
            dil_kernel = []
            for channel in kernel:
                dil_channel = np.zeros((dil_ker_h, dil_ker_w))
                for row in range(len(channel)):
                    for col in range(len(channel[0])):
                        dil_channel[self.dilation*row][self.dilation*col] = channel[row][col]
                dil_kernel.append(dil_channel.tolist())
            dil_kernels.append(dil_kernel)
        kernels, self.kernel_size = dil_kernels, (dil_ker_h, dil_ker_w)
        self.verboseprint('*** dilated kernels ***')
        self.verboseprint('kernels: {}, dilation factor: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, self.dilation, int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]))
        for k in range(self.out_channels):
            self.verboseprint('kernel {}'.format(k))
            self.verboseprint(kernels[k])
        self.verboseprint('\n')
        
        ''' compute output volume from the input and kernel parameters '''
        _input_n, _, _input_h, _input_w = _input.shape
        out_n = int(_input_n)
        out_c = int(self.out_channels)
        out_h = int((_input_h - self.kernel_size[0])/self.stride) + 1
        out_w = int((_input_w - self.kernel_size[1])/self.stride) + 1
        output = np.zeros([out_n, out_c, out_h, out_w])
        
        ''' parse through every element of the output and compute the convolution value for that element '''
        for b in range(out_n):
            for k in range(out_c):
                for h in range(out_h):
                    for w in range(out_w):
                        # convolve kernel over the input slices
                        self.debugprint('kernel indices, image indices')
                        self.debugprint('[n, c, h, w]', '[n, c, h, w]')
                        convol_sum = 0
                        ker_c = int(self.in_channels / self.groups)
                        ker_h = self.kernel_size[0]
                        ker_w = self.kernel_size[1]
                        for c_ker in range(ker_c):
                            for h_ker in range(ker_h):
                                for w_ker in range(ker_w):
                                    # if self.groups == 1:
                                        # self.debugprint([k, c_ker, h_ker, w_ker], [b, c_ker, h_ker + self.stride * h, w_ker + self.stride * w])
                                        # convol_sum += kernels[k][c_ker][h_ker][w_ker] * _input[b][c_ker][h_ker + self.stride*h][w_ker + self.stride*w] # works only when `groups` == 1
                                    # elif self.groups == self.in_channels:
                                        # self.debugprint([k, c_ker, h_ker, w_ker], [b, k, h_ker + self.stride * h, w_ker + self.stride * w])
                                        # convol_sum += kernels[k][c_ker][h_ker][w_ker] * _input[b][k][h_ker + self.stride * h][w_ker + self.stride * w] # works only when `groups` == in_channels
                                    # else:
                                        # self.debugprint([k, c_ker, h_ker, w_ker], [b, c_ker if k < self.groups else c_ker + ker_c, h_ker + self.stride * h, w_ker + self.stride * w])
                                        # convol_sum += kernels[k][c_ker][h_ker][w_ker] * _input[b][c_ker if k < self.groups else c_ker + ker_c][h_ker + self.stride * h][w_ker + self.stride * w] # works in some cases when 1 < `groups` < in_channels
                                    self.debugprint([k, c_ker, h_ker, w_ker], [b, c_ker + (ker_c * int(np.floor(k / (self.out_channels / self.groups)))), h_ker + self.stride * h, w_ker + self.stride * w])
                                    convol_sum += kernels[k][c_ker][h_ker][w_ker] * _input[b][c_ker + (ker_c * int(np.floor(k / (self.out_channels / self.groups))))][h_ker + self.stride * h][w_ker + self.stride * w] # works for all 1 <= `groups` <= in_channels
                        self.debugprint('\n')
                        output[b, k, h, w] += convol_sum
        self.verboseprint('*** Conv2DPytTest output ***')
        output_shape = output.shape
        self.verboseprint('output batches: {}, ouput channels: {}, output height: {}, output weight: {}'.format(output_shape[0], output_shape[1], output_shape[2], output_shape[3]))
        assert((out_n, out_c, out_h, out_w) == output_shape)
        self.verboseprint(output)
        self.verboseprint('\n')
        return output

### Standalone test (random kernel, random input)

In [81]:
debug = False # DO NOT CHANGE THIS while using Conv2DPytTestGroups - TODO: modify this flag to `test`

in_channels = 6 # input channels
out_channels = 4 # output channels
kernel_size = (2, 2) # kernel size

padding = 1 # padding (optional)
stride = 2 # stride (optional)
dilation = 1 # dilation factor (optional)
groups = 2 # groups (optional)

in_batches = 2 # input batches
in_h = 4 # input height
in_w = 4 # input weight

_input = np.random.rand(in_batches, in_channels, in_h, in_w) # define a random image based on the input parameters
kernels = []
for k in range(out_channels):
    kernel = np.random.rand(int(in_channels / groups), kernel_size[0], kernel_size[1]) # define a random kernel based on the kernel parameters
    kernels.append(kernel)

### Debug cell

In [83]:
# get Conv2DPytTestGroups output with the random inputs

conv2dpyttestgroups = Conv2DPytTestGroups(in_channels, out_channels, kernel_size, stride = stride, padding = padding, dilation = dilation, groups = groups, debug = debug, verbose = False) # call an instance of the class with the input parameters 
_output = conv2dpyttestgroups.forward(_input, kernels) # perform convolution
print("*** Conv2DPytTestGroups output ***")
print(_output)

*** Conv2DPytTestGroups output ***
[[[[1.10192836]
   [1.51187638]
   [1.02969919]
   [0.64293919]]

  [[1.26775422]
   [1.8186179 ]
   [2.22417617]
   [1.71097682]]]


 [[[1.1597717 ]
   [0.74006726]
   [0.63493165]
   [0.66291379]]

  [[1.05911239]
   [0.80990874]
   [1.26669659]
   [1.68991502]]]


 [[[1.03753299]
   [1.53259776]
   [0.67928173]
   [0.54115662]]

  [[0.94056286]
   [2.03424528]
   [1.26534445]
   [1.24467719]]]


 [[[0.73065443]
   [1.41083697]
   [0.70651766]
   [0.51291586]]

  [[1.05819814]
   [2.11853918]
   [1.63883502]
   [0.91626343]]]


 [[[1.46247271]
   [0.91815224]
   [0.96290508]
   [1.0433927 ]]

  [[1.24622601]
   [0.86230355]
   [1.81638351]
   [2.00142631]]]]


In [88]:
# get PyTorch output with the same random inputs as above

import torch

x = torch.DoubleTensor(_input)
weights = torch.stack([torch.DoubleTensor(kernel) for kernel in kernels])
output = torch.nn.functional.conv2d(x, weights, stride = stride, padding = padding, dilation = dilation, groups = groups)
print("*** PyTorch output ***")
print(output)

*** PyTorch output ***
tensor([[[[1.1019],
          [1.5119],
          [1.0297],
          [0.6429]],

         [[1.2678],
          [1.8186],
          [2.2242],
          [1.7110]]],


        [[[1.1598],
          [0.7401],
          [0.6349],
          [0.6629]],

         [[1.0591],
          [0.8099],
          [1.2667],
          [1.6899]]],


        [[[1.0375],
          [1.5326],
          [0.6793],
          [0.5412]],

         [[0.9406],
          [2.0342],
          [1.2653],
          [1.2447]]],


        [[[0.7307],
          [1.4108],
          [0.7065],
          [0.5129]],

         [[1.0582],
          [2.1185],
          [1.6388],
          [0.9163]]],


        [[[1.4625],
          [0.9182],
          [0.9629],
          [1.0434]],

         [[1.2462],
          [0.8623],
          [1.8164],
          [2.0014]]]], dtype=torch.float64)


In [89]:
# compare outputs of conv-Numpy and PyTorch
print(torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output))) # need to round the output due to precision difference

True


### Extensive tests (random kernel, random input)

In [7]:
def valid_params(num_tests):
    # generates `num_samples` number of valid input and kernel parameters 
    params_list = []
    sample_count = 0
    while sample_count < num_tests:
        in_channels = np.random.randint(20) + 1 # input channels
        out_channels = np.random.randint(20) + 1 # output channels
        
        ker_h = np.random.randint(20) + 1
        ker_w = np.random.randint(20) + 1
        kernel_size = (ker_h, ker_w) # kernel size
        
        padding = np.random.randint(10) + 1 # padding (optional)
        stride = np.random.randint(5) + 1 # stride (optional)
        dilation = np.random.randint(10) + 1 # dilation factor (optional)
        groups = np.random.randint(in_channels) + 1 # groups (optional)
        
        in_batches = np.random.randint(5) + 1 # input batches
        in_h = np.random.randint(30) + 5 # input height
        in_w = np.random.randint(30) + 5 # input weight
    
        ker_h_flag, ker_w_flag, out_h_flag, out_w_flag, in_group_flag, out_group_flag = True, True, True, True, True, True
        
        if in_h + 2 * padding < dilation * (ker_h - 1) + 1: # check if (dilated) ker_h is valid
            ker_h_flag = False
        if in_w + 2 * padding < dilation * (ker_w - 1) + 1: # check if (dilated) ker_w is valid
            ker_w_flag = False
        if ((in_h + 2 * padding - (dilation * (ker_h - 1) + 1)) / stride) + 1 < 0: # check if out_h is valid
            out_h_flag = False
        if ((in_w + 2 * padding - (dilation * (ker_w - 1) + 1)) / stride) + 1 < 0: # check if out_w is valid
            out_w_flag = False
        if  in_channels % groups != 0: # check if groups is valid
            in_group_flag = False
        if out_channels % groups != 0: # check if groups is valid
            out_group_flag = False
        
        if ker_h_flag and ker_w_flag and out_h_flag and out_w_flag and in_group_flag and out_group_flag:
            params_list.append({'in_channels': in_channels, 'out_channels': out_channels, 'kernel_size': kernel_size,
                          'padding': padding, 'stride': stride, 'dilation': dilation, 'groups': groups, 'in_batches': in_batches,
                          'in_h': in_h, 'in_w': in_w})
            sample_count += 1
    return params_list

In [90]:
# for loop sweeping different input parameters and testing the outputs of Conv2DPytTestGroups and PyTorch
from tqdm import tqdm

num_tests = 200 # only test parameter to be varied
num_passed = 0
params_list = valid_params(num_tests)
print('Number of tests: {}\n\n'.format(len(params_list)))

for i, params in enumerate(tqdm(params_list)):
    print('Test: {}\nParams: {}'.format(i, params))
    debug = False # DO NOT CHANGE THIS while using Conv2DPytTest - TODO: modify this flag to `test`

    in_channels = params['in_channels'] # input channels
    out_channels = params['out_channels'] # output channels
    
    kernel_size = params['kernel_size'] # kernel size

    padding = params['padding'] # padding (optional)
    stride = params['stride'] # stride (optional)
    dilation = params['dilation'] # dilation factor (optional)
    groups = params['groups'] # groups (optional)
    
    in_batches = params['in_batches'] # input batches
    in_h = params['in_h'] # input height
    in_w = params['in_w'] # input weight
    
    _input = np.random.rand(in_batches, in_channels, in_h, in_w) # define a random image based on the input parameters
    kernels = []
    for k in range(out_channels):
        kernel = np.random.rand(int(in_channels / groups), kernel_size[0], kernel_size[1]) # define a random kernel based on the kernel parameters
        kernels.append(kernel)
    
    try:
        # get Conv2DPytTest output with the random inputs
        conv2dpyttestgroups = Conv2DPytTestGroups(in_channels, out_channels, kernel_size, stride = stride, padding = padding, dilation = dilation, groups = groups, debug = debug, verbose = False) # call an instance of the class with the input parameters 
        _output = conv2dpyttestgroups.forward(_input, kernels) # perform convolution

        # get PyTorch output with the same random inputs as above
        x = torch.DoubleTensor(_input)
        weights = torch.stack([torch.DoubleTensor(kernel) for kernel in kernels])
        output = torch.nn.functional.conv2d(x, weights, stride = stride, padding = padding, dilation = dilation, groups = groups)
        
    except Exception as e:
        print(e)
        print('Result: False\n\n') # treating exception as a failed test
        continue
    
    # compare outputs of conv-Numpy and PyTorch
    result = torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output)) # need to round the output due to precision difference
    print('Result: {}\n\n'.format(result))
    if result:
        num_passed += 1

print('{} out of {} ({}%) tests passed'.format(num_passed, num_tests, float(100 * num_passed / num_tests)))
    

Number of tests: 200




  0%|                                                   | 0/200 [00:00<?, ?it/s]

Test: 0
Params: {'in_channels': 14, 'out_channels': 4, 'kernel_size': (7, 5), 'padding': 9, 'stride': 5, 'dilation': 2, 'groups': 1, 'in_batches': 4, 'in_h': 27, 'in_w': 24}


  0%|▏                                          | 1/200 [00:06<22:56,  6.92s/it]

Result: True


Test: 1
Params: {'in_channels': 4, 'out_channels': 4, 'kernel_size': (20, 11), 'padding': 8, 'stride': 3, 'dilation': 1, 'groups': 1, 'in_batches': 1, 'in_h': 23, 'in_w': 28}


...

Result: True


Test: 198
Params: {'in_channels': 3, 'out_channels': 2, 'kernel_size': (8, 2), 'padding': 10, 'stride': 5, 'dilation': 3, 'groups': 1, 'in_batches': 3, 'in_h': 29, 'in_w': 24}


100%|████████████████████████████████████████▊| 199/200 [24:15<00:20, 20.99s/it]

Result: True


Test: 199
Params: {'in_channels': 10, 'out_channels': 4, 'kernel_size': (6, 8), 'padding': 8, 'stride': 3, 'dilation': 4, 'groups': 1, 'in_batches': 1, 'in_h': 17, 'in_w': 18}


100%|█████████████████████████████████████████| 200/200 [24:16<00:00,  7.28s/it]

Result: True


200 out of 200 (100.0%) tests passed


### Modular code for methods in `forward()`

### Modular code for `forward()`

### Old `Conv2D` code  for reference